# 2022.8#1
本周我们复现的模型为[3-D Deep Learning Approach for Remote Sensing Image Classification Amina Ben Hamida, Alexandre Benoit, Patrick Lambert, Chokri Ben Amar IEEE TGRS, 2018](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8344565)中介绍的模型，一个基于 3D CNN 的高光谱影像分类。

我们参考的实现使用 visdom 进行可视化，tqdm 展示进度条。我们在 Jupyter 内重新实现本来应该使用 matplotlib 进行可视化， 本次实现略过了可视化部分。

## 数据处理
第一部分，设置各种超参数并从文件中读取数据。这部分的变量都是可以调整的。

In [25]:
import torch
import numpy as np
from scipy import io

# 参数集
params = {
    'dataset': 'Pavia',
    'training_sample': 0.1, # 用于训练的数据，其余用于测试
    'epoch': 2,
    'batch_size': 100,
    'lr': 0.001,
}

# 使用 GPU
CUDA_DEVICE = torch.device('cuda:0')

# 加载数据集，这里使用Pavia
path = './dataset/Pavia/'
img = io.loadmat(path + 'Pavia.mat')['pavia']
gt = io.loadmat(path + 'Pavia_gt.mat')['pavia_gt']
label_values = [
    "Undefined",
    "Water",
    "Trees",
    "Asphalt",
    "Self-Blocking Bricks",
    "Bitumen",
    "Tiles",
    "Shadows",
    "Meadows",
    "Bare Soil",
]
# 归一化
img = np.asarray(img, dtype="float32")
img = (img - np.min(img)) / (np.max(img) - np.min(img))

num_classes = len(label_values)
num_bands = img.shape[-1]

## 分离训练集和测试集
传感器提供的数据是一个整体，需要手动将数据（ground truth部分）分为训练集和测试集，用空值填充其余部分（相当于 Undefined 类）。scikit-learn 包提供了一个将列表按指定的比例或数量分离的工具。

In [17]:
import sklearn.model_selection
# 后续需要利用该函数进行交叉验证
def sample_gt(gt, train_size):
    indices = np.nonzero(gt)
    X = list(zip(*indices)) # x,y features
    y = gt[indices].ravel() # classes
    train_gt = np.zeros_like(gt)
    test_gt = np.zeros_like(gt)
    if train_size > 1:
       train_size = int(train_size)

    train_indices, test_indices = sklearn.model_selection.train_test_split(X, train_size=train_size, stratify=y)
    train_indices = [list(t) for t in zip(*train_indices)]
    test_indices = [list(t) for t in zip(*test_indices)]
    train_gt[tuple(train_indices)] = gt[tuple(train_indices)]
    test_gt[tuple(test_indices)] = gt[tuple(test_indices)]
    return train_gt, test_gt

train_gt, test_gt = sample_gt(gt, params['training_sample'])
"在 {} 个样本中选择了 {} 个" .format(np.count_nonzero(gt), np.count_nonzero(train_gt))

'在 148152 个样本中选择了 14815 个'

## 模型创建

In [18]:
from torch import nn, optim
from torch.nn import init
import torch.nn.functional as F

class Module(nn.Module):
    # 初始化模型参数
    @staticmethod
    def weight_init(m):
        if isinstance(m, (nn.Linear, nn.Conv3d)):
            init.kaiming_normal_(m.weight)
            init.zeros_(m.bias)

    def __init__(self, input_channels, n_classes, patch_size=5, dilation=1):
        super(Module, self).__init__()
        # 第一层卷积层，（3，3，3）卷积核，步长为1，20个节点
        self.patch_size = patch_size
        self.input_channels = input_channels
        dilation = (dilation, 1, 1)

        if patch_size == 3:
            self.conv1 = nn.Conv3d(1, 20, (3, 3, 3), stride=(1, 1, 1), dilation=dilation, padding=1)
        else:
            self.conv1 = nn.Conv3d(1, 20, (3, 3, 3), stride=(1, 1, 1), dilation=dilation, padding=0)

        # 第二层池化层，（1，1，3）一维卷积核，步长为2以在光谱方向上减少维度
        self.pool1 = nn.Conv3d(20, 20, (3, 1, 1), dilation=dilation, stride=(2, 1, 1), padding=(1, 0, 0))

        # 然后是前两层的重复，节点数量为35
        self.conv2 = nn.Conv3d(20, 35, (3, 3, 3), dilation=dilation, stride=(1, 1, 1), padding=(1, 0, 0))
        self.pool2 = nn.Conv3d(35, 35, (3, 1, 1), dilation=dilation, stride=(2, 1, 1), padding=(1, 0, 0))

        # 进行两次一维卷积
        self.conv3 = nn.Conv3d(35, 35, (3, 1, 1), dilation=dilation, stride=(1, 1, 1), padding=(1, 0, 0))
        self.conv4 = nn.Conv3d(35, 35, (2, 1, 1), dilation=dilation, stride=(2, 1, 1), padding=(1, 0, 0))

        # self.dropout = nn.Dropout(p=0.5)

        self.features_size = self._get_final_flattened_size()
        
        # 最后是单独的全连接层
        self.fc = nn.Linear(self.features_size, n_classes)

        self.apply(self.weight_init)

    # 计算可变输入展开后的列表长度
    def _get_final_flattened_size(self):
        with torch.no_grad():
            x = torch.zeros(
                (1, 1, self.input_channels, self.patch_size, self.patch_size)
            )
            x = self.pool1(self.conv1(x))
            x = self.pool2(self.conv2(x))
            x = self.conv3(x)
            x = self.conv4(x)
            _, t, c, w, h = x.size()
        return t * c * w * h

    def forward(self, x):
        # 全部使用ReLU激活函数
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, self.features_size)
        # x = self.dropout(x)
        x = self.fc(x)
        return x


net = Module(num_bands, num_classes)
net = net.to(CUDA_DEVICE)
# 随机梯度下降
optimizer = optim.SGD(net.parameters(), params['lr'], weight_decay=0.0005)
# 交叉熵损失
criterion = nn.CrossEntropyLoss(weight=torch.ones(num_classes).to(CUDA_DEVICE))

## 数据加载器
此时的数据仍为以 narray 形式存储的数集，需要实现一个继承类以用 Pytorch 提供的 api 进行 batch 级的数据加载。

In [19]:
from torch.utils import data

train_gt, val_gt = sample_gt(train_gt, 0.95)
class Dataset(data.Dataset):

    def __init__(self, data, gt, **params):
        super(Dataset, self).__init__()
        self.data = data
        self.label = gt
        self.name = params["dataset"]
        self.patch_size = 5
        mask = np.ones_like(gt)
        x_pos, y_pos = np.nonzero(mask)
        p = self.patch_size // 2
        self.indices = np.array(
            [
                (x, y)
                for x, y in zip(x_pos, y_pos)
                if x > p and x < data.shape[0] - p and y > p and y < data.shape[1] - p
            ]
        )
        self.labels = [self.label[x, y] for x, y in self.indices]
        np.random.shuffle(self.indices)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        x, y = self.indices[i]
        x1, y1 = x - self.patch_size // 2, y - self.patch_size // 2
        x2, y2 = x1 + self.patch_size, y1 + self.patch_size

        data = self.data[x1:x2, y1:y2]
        label = self.label[x1:x2, y1:y2]

        # Copy the data into numpy arrays (PyTorch doesn't like numpy views)
        data = np.asarray(np.copy(data).transpose((2, 0, 1)), dtype="float32")
        label = np.asarray(np.copy(label), dtype="int64")

        # 将数据加载为 Tensor
        data = torch.from_numpy(data)
        label = torch.from_numpy(label)
        label = label[self.patch_size // 2, self.patch_size // 2]

        # Add a fourth dimension for 3D CNN
        if self.patch_size > 1:
            # Make 4D data ((Batch x) Planes x Channels x Width x Height)
            data = data.unsqueeze(0)
        return data, label

train_dataset = Dataset(img, train_gt, **params)
train_loader = data.DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
val_dataset = Dataset(img, val_gt, **params)
val_loader = data.DataLoader(val_dataset, batch_size=params['batch_size'])

## 进行训练

In [30]:
for e in range(1, params['epoch'] + 1):
    net.train()
    iter_ = 0
    base2 = 1

    for train_data, target in train_loader:
        train_data, target = train_data.to(CUDA_DEVICE), target.to(CUDA_DEVICE)

        optimizer.zero_grad()
        output = net(train_data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        # 输出损失
        if iter_ == base2:
            base2 *= 2
            print(loss.item())
        iter_ += 1

0.01686476171016693
0.07508590072393417
0.03868788480758667
0.2064131498336792
0.22261665761470795
0.04249861091375351
0.1363358497619629
0.10841484367847443
0.052877526730298996
0.08296363800764084
0.1536632478237152
0.019730927422642708
0.07960836589336395
0.1741037517786026
0.14908188581466675
0.05638481676578522
0.016952678561210632
0.06932863593101501
0.016139334067702293
0.012532655149698257
0.08281221240758896
0.02270493470132351
0.05635599046945572
0.1597495973110199
0.04146759957075119
0.07195183634757996


## 结果测试
暂时使用 visdom 可视化的结果截图。

![train loss 和 validation accuracy](file:///d%3A/repos/Hyperspectral/static/QQ%E6%88%AA%E5%9B%BE20220808152636.jpg)

![混淆矩阵](file:///d%3A/repos/Hyperspectral/static/QQ%E6%88%AA%E5%9B%BE20220808152651.jpg)

![预测结果](file:///d%3A/repos/Hyperspectral/static/QQ%E6%88%AA%E5%9B%BE20220808152720.jpg)